<a href="https://colab.research.google.com/github/ValiantChiu/Computer_Vision_AICSP/blob/main/%E9%9B%BB%E5%95%86%E5%B9%B3%E5%8F%B0%E5%AF%A6%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1

## Install packages

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

## Import Packages

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

## Query Data

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

url = 'https://www.books.com.tw/web/annual100'
browser.get(url)

In [ ]:
books_raw = BeautifulSoup(browser.page_source).\
find("div", {"class": "mod_d type02_m100 clearfix"}).\
find("div", {"class": "table-container"})
books = books_raw.find_all('a')
prices = books_raw.find_all('b')[2::3]

## Clean the data

In [ ]:
get_title_link = lambda x: [str(x).split('">')[1].replace("</a>",""), str(x).split('">')[0].replace('<a href="',"")]

In [ ]:
with_book = lambda x: True if 'a href="https://www.books.com.tw/products' in str(x) and 'img' not in str(x) else False

In [ ]:
books_list = [get_title_link(book) for book in books if with_book(book)]

In [ ]:
books_titles = [book[0] for book in books_list]
books_links = [book[1] for book in books_list]
books_prices = [str(price).replace('</b>','').replace('<b>','') for price in prices]

In [ ]:
books_dict = dict(zip(range(1,101), zip(books_titles, books_prices, books_links)))

## User session

In [ ]:
def check_input(customer_book_start_range, customer_book_end_range):
    def check_input(start, end):
        not_digit, out_of_range, wrong_sequence = False, False, False
        not_digit = any([not start.isdigit(),not end.isdigit()])
        if not not_digit:
            out_of_range = eval(start) >=100 or eval(end) > 100 or eval(start) < 1 or eval(end) < 1
            wrong_sequence = eval(start) > eval(end)
        return any([not_digit, out_of_range, wrong_sequence])
    while check_input(customer_book_start_range, customer_book_end_range):
        print ("Please check your format again(only number),\nstart must smaller than end,\nand not greater than 100 and smaller than 1 : ")
        customer_book_start_range = input('start from:')
        customer_book_end_range = input('end to:')
    else:
        return (customer_book_start_range, customer_book_end_range)

In [ ]:
def show_books(start, end, books_dict):
    total_price = 0
    cart = []
    for i in range(int(start), int(end) + 1):
        title, price, link = books_dict[i]
        print("\nThe Books List: ")
        print(f'{i} { title },\n    Price: { price }, \n    Link: { link }\n')
        total_price += int(price)
        cart.append(title)
    return total_price, cart

In [ ]:
customer_name = input('Welcome to Ｐｙｔｈｏｎ　ＡＩ book store,\nPlease input your name:')
customer_budget = eval(input('Please input your book budget:'))
customer_book_start_range = input(f'\n\nHello, {customer_name}, we have top 100 books this year,\nplease choose the rank range you want to know\nstart from:')
customer_book_end_range = input('end to:')
customer_book_start_range, customer_book_end_range = check_input(customer_book_start_range, customer_book_end_range)

is_buy = False
is_exit = False
is_first_visit = True
customer_car = []
while not is_buy or not is_exit:
    if is_first_visit:
        total_price, cart = show_books(customer_book_start_range, customer_book_end_range, books_dict)
        is_buy = input(f'Total price is: {total_price}\nBuy these books?(Y/N)').upper().strip() == 'Y'
        budget_remained = customer_budget - total_price
        is_first_visit = False
    else:
        print('Search new books')
        customer_book_start_range = input('start from:')
        customer_book_end_range = input('end to:')
        customer_book_start_range, customer_book_end_range = check_input(customer_book_start_range, customer_book_end_range)
        total_price, cart = show_books(customer_book_start_range, customer_book_end_range, books_dict)
        budget_remained = customer_budget - total_price
        is_buy = input(f'Total price is: {total_price}\nBuy these books?(Y/N)').upper().strip() == 'Y'

    if budget_remained >= 0 and is_buy :
        print(f'budget remained { budget_remained }')
        customer_budget = customer_budget - total_price
        customer_car.append(cart)
        is_exit = input(f'You already bought { customer_car }\nExit store?(Y/N)').upper() == 'Y'
        if is_exit: 
            print(f'\n\nGoodbye!{ customer_name }.')
    else:
        if budget_remained < 0 and is_buy:
            print(f"Unfortunately, you don't have enough money.")
            is_exit = input(f'You already bought { customer_car }\nExit store?(Y/N)').upper() == 'Y'
            if is_exit: 
                print(f'\n\nGoodbye!{ customer_name }.')
                break;